# How to create your own API

- How can we make different softwares communicate? All we need is an API

An API (**Application Programming Interface**) defines the types of requests that a software can make to another

Requests are classified into

- GET/POST/PUT/PATCH/DELETE 

## Example

Let's consider a toy model in which we have data on firms, CEOs, and their compensation. 

In [1]:
import pandas as pd

ceos = [{"Name" : "Doug McMillion", "Age" : 55},
        {"Name" : "Fu Chengyu", "Age" : 70}, 
        {"Name" : "Andy Jassy", "Age" : 53}, 
        {"Name" : "Zhou Jiping", "Age" : 69}, 
        {"Name" : "Tim Cook", "Age" : 60}]

companies = [{"Name" : "Walmart Inc.", "Ticker" : "WMT", "Revenue (billions)" : 542, "CEO" : "Doug McMillion"},
             {"Name" : "China Petroleum and Chemical Corp.", "Ticker" : "SNP", "Revenue (billions)" : 355.8, "CEO" : "Fu Chengyu"},
             {"Name" : "Amazon.com Inc", "Ticker" : "AMZN", "Revenue (billions)" : 321.8, "CEO" : "Jeff Bezos"}, 
             {"Name" : "PetroChina Co. Ltd", "Ticker" : "PTR", "Revenue (billions)" : 320, "CEO" : "Zhou Jiping"}, 
             {"Name" : "Apple Inc.", "Ticker" : "AAPL", "Revenue (billions)" : 273.9, "CEO" : "Tim Cook"}]

df_ceos = pd.DataFrame.from_dict(ceos)
print(df_ceos.head())

df_companies = pd.DataFrame.from_dict(companies)
print(df_companies)

             Name  Age
0  Doug McMillion   55
1      Fu Chengyu   70
2      Andy Jassy   53
3     Zhou Jiping   69
4        Tim Cook   60
                                 Name Ticker  Revenue (billions)  \
0                        Walmart Inc.    WMT               542.0   
1  China Petroleum and Chemical Corp.    SNP               355.8   
2                      Amazon.com Inc   AMZN               321.8   
3                  PetroChina Co. Ltd    PTR               320.0   
4                          Apple Inc.   AAPL               273.9   

              CEO  
0  Doug McMillion  
1      Fu Chengyu  
2      Jeff Bezos  
3     Zhou Jiping  
4        Tim Cook  


## Basic REST API
- Representational state transfer
- An API requires an internet connection or at least a localhost
- We will use Python's Flask library to create the API
- Requests are done via HTTP requests (urls)

In [2]:
!pip install flask
!pip install flask_restful
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
from flask_ngrok import run_with_ngrok
import ast

app = Flask("My App")
api = Api(app)

## Endpoints

We have 2 endpoints, CEOS and Companies. If our API were located at `www.mycoolapi.com`, communications would be done by accessing `www.mycoolapi.com/ceos` and `www.mycoolapi.com/companies`

In [4]:
# Create an endpoint as a Python class

class Ceos(Resource):
    pass

class Companies(Resource):
    pass

api.add_resource(Ceos, '/ceos')
api.add_resource(Companies, '/companies')

In [5]:
# add some html so there is something to look at
@app.route("/")
def home():
    return "<h1>Coolest API</h1>"

Flask needs to know that this class is an endpoint for our API, and so we pass Resource in with the class definition.

Inside the class, we include our HTTP methods (GET, POST, DELETE, etc.).

Run the API (include `if __name__ == '__main__':` before the `app.run()` command when creating a .py file)

In [6]:
# use ngrok
!pip install pyngrok
# NOTE: For the project use your own authtoken
!ngrok authtoken 1mvIJph4MZBcieB83167SLwMz2Q_4vPTn8XZfX9UnV6uqTsAr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.8 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=ec0fc9eaec22848ab0f528095f45a65ad11f06bc80bd5186961389af6f400142
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
if __name__ == '__main__':
  
  run_with_ngrok(app)
  app.run()


 * Serving Flask app "My App" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5731-34-125-72-122.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:40:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:40:34] "GET /favicon.ico HTTP/1.1" 404 -


## Writing API methods

The GET method is the simplest. We return all data stored in the dataframe wrapped inside a dictionary, like so:

In [8]:
class Ceos(Resource):
    
    def get(self):
        ceos = [df_ceos.iloc[i].to_dict() for i in range(len(df_ceos))]
        return  ceos , 200 # The 200 is the standard way to say everything is ok

class Companies(Resource):
    
    def get(self):
        companies = [df_companies.iloc[i].to_dict() for i in range(len(df_companies))]
        return  companies , 200 # The 200 is the standard way to say everything is ok

api = Api(Flask("My App"))
api.add_resource(Ceos, '/ceos')
api.add_resource(Companies, '/companies')

## Customize your GET methods

GET methods can be customized to obtain only a subset of the data using **keywords**. These **keywords** are passed through the url. E.g. 

`www.mycoolapi.com/companies?ticker=aapl`

Ask specific companies or ranges of profits

In [26]:
class Companies(Resource):

    def get(self):

        parser = reqparse.RequestParser()

        parser.add_argument('ticker', required = False) # If the ticker is mandatory for the request, change to True
        parser.add_argument('min_revenue', required=False)

        args = parser.parse_args()

        # Args is a dictionary
        data = df_companies
        if args['ticker'] is not None:
            data = df_companies[df_companies['Ticker'] == args['ticker']]

        if args['min_revenue'] is not None:
            data = df_companies[df_companies['Revenue (billions)'] >= args['ticker']]

        companies = [data.iloc[i].to_json() for i in range(len(data))]

        return companies, 200
        

Request CEOS older than some age

In [27]:
class Ceos(Resource):

    def get(self):

        parser = reqparse.RequestParser()
        parser.add_argument('min_age', required=False)

        args = parser.parse_args()

        # Args is a dictionary
        data = df_ceos
        if args['min_age'] is not None:
            data = df_ceos[df_ceos['Age'] == args['min_age']]

        ceos = [data.iloc[i].to_json() for i in range(len(data))]

        return ceos, 200

In [28]:
app=Flask("My App")
api = Api(app)
api.add_resource(Ceos, '/ceos')
api.add_resource(Companies, '/companies')

@app.route("/")
def home():
    return "<h1>Coolest API</h1>"


In [29]:

if __name__ == '__main__':
  
  run_with_ngrok(app)
  app.run()

 * Serving Flask app "My App" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ab13-34-125-72-122.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:52:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:52:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:52:19] "GET /ceos HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:52:28] "GET /companies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 16:52:47] "GET /companies?ticker=AAPL HTTP/1.1" 200 -
